In [1]:
#setup imports and make ure the files we care about exist
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os.path

#custom modules
import feature_extraction
import unsw_nb15_dataset
import generator_discriminator
import train

from importlib import reload #allow us to reload custom modules any time we like with modifications
reload(feature_extraction)
reload(unsw_nb15_dataset)
reload(generator_discriminator)
reload(train)

#fe.test_cases()

<module 'train' from '/home/jaywalker/MachineLearning/PacketGAN/train.py'>

In [ ]:
#first, load the dataset
data_set = unsw_nb15_dataset.UNSW_NB15(['/home/jaywalker/MachineLearning/PacketGAN/UNSW-NB15_1_clean.csv'],
                                       sequence_length=5, 
                                       transform=feature_extraction.build_feature_sequence_tensor)

#pick out the attack examples
data_set.use_only_category('DoS'); #in this case just look at DoS attack samples

#convert attack examples into time windows for training
#DONE by the UNSW_NB15 class at load-time

#convert each window into appropriate feature tensor
#DONE by the transform function passed to the UNSW_NB15 class at load-time

#train GAN to replace masked packets
#by iterating over the dataset for some # of epochs
#TODO make a python file to contain the network class definitions and training funciton

data_loader = DataLoader(data_set, batch_size=1,
                        shuffle=True, num_workers=4)

data_example = None
for count, data in enumerate(data_loader):
    data_example = data
    if count == 0:
        break
        
G = generator_discriminator.Generator(data_example.shape[2], data_example.shape[2] * 2, 30, data_example.shape[2], torch.sigmoid)
D = generator_discriminator.Discriminator(data_example.shape[2], data_example.shape[2] * 2, 1, torch.sigmoid)
gen, g_losses, df_losses, g_stats, df_stats = train.train(G,D,data_loader)

#plot instantaneous losses
plt.plot(range(len(df_losses)), df_losses, 'r-', range(len(g_losses)), g_losses, 'g-')
plt.show()

#plot loss averages
plt.plot(range(len(df_stats.get_averages())), df_stats.get_averages(), 'r-', range(len(g_stats.get_averages())), g_stats.get_averages(), 'g-')
plt.show()



Training has started...


/home/jaywalker/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2016: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([1, 5, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch:  0
D Real Error:  0.6842836737632751
D Fake Error:  0.570070743560791
G Error:  1.6903883218765259
Remaining time: 0:17:27.254741
Epoch:  1
D Real Error:  0.5898365378379822
D Fake Error:  0.4941752552986145
G Error:  1.869036316871643
Remaining time: 0:19:41.868359
Epoch:  2
D Real Error:  0.6052052974700928
D Fake Error:  0.7895750999450684
G Error:  1.7958694696426392
Remaining time: 0:17:33.142356
Epoch:  3
D Real Error:  0.6210771203041077
D Fake Error:  0.6834601163864136
G Error:  0.7495638132095337
Remaining time: 0:16:31.522563
Epoch:  4
D Real Error:  0.6504245400428772
D Fake Error:  0.656497597694397
G Error:  0.7433679699897766
Remaining time: 0:20:34.244055
Epoch:  5
D Real Error:  0.643572986125946
D Fake Error:  0.729261577129364
G Error:  0.6605849266052246
Remaining time: 0:14:01.220212
Epoch:  6
D Real Error:  0.7229766845703125
D Fake Error:  0.6641817092895508
G Error:  0.721723735332489
Remaining time: 0:12:03.430074
Epoch:  7
D Real Error:  0.7056003808975

In [ ]:
#evaluate results!
print("Real data: ", decode_feature_sequence_tensor(data_set[0]), end="\n\n")

#note here that "gen" is the trained generator returned from the training function above
print("Fake data: ", decode_feature_sequence_tensor(G(data_set[0]).detach()))